In [ ]:
import sys
import os
import findspark
findspark.init()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
import requests
from urllib.request import urlopen
import json

In [ ]:
import spark
import pyspark
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName('capstone_project').getOrCreate()

In [ ]:
 user = os.getenv("user", default=None)
 password = os.getenv("password", default=None)

In [ ]:
url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'

In [ ]:
try:
    response = requests.get(url,timeout=3)
    response.raise_for_status()
except requests.exceptions.RequestException as err:
    print ("Oops: Something Else",err)
except requests.exceptions.HTTPError as errh:
    print ("Http Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)     
print(response.status_code)

In [ ]:
#loan_data = json.loads(response.text)
loan_datafile = os.path.join("files", "loan_datafile.json")
loan_data = response.json()
#print(loan_data)
with open(loan_datafile,'w') as f:
    json.dump(loan_data,f)
#print(loan_data)

In [ ]:
loan_dataset = spark.read.format('json').load(loan_datafile)

In [ ]:
loan_dataset.printSchema()

In [ ]:
loan_dataset.show()

In [ ]:
f.close()

In [ ]:
# ' DROP TABLE `creditcard_capstone`.`cdw_sapp_loan_application`;'

In [ ]:
loan_dataset.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.cdw_sapp_loan_application") \
.option("user", user) \
.option("password", password) \
.save()

In [ ]:
spark.stop()

In [ ]:
#DROP TABLE `creditcard_capstone`.`cdw_sapp_loan_application`;